In [2]:
# For adding  populations information in populations table in sql database

import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from keys import sql_password, API_key_city_info


#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = sql_password
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)
    
def create_populations_df():
    connection_string = create_connection_string()
    cities_from_sql = fetch_cities_data(connection_string)
    #cities = cities_from_sql["city_name"]

    population = []
    year_retrieved =[]
    city_id = []
    for index, row in cities_from_sql.iterrows():
        #https://api.api-ninjas.com/v1/city?name=San Francisco
        response = requests.get(f"https://api.api-ninjas.com/v1/city?name={row['city_name']}", headers={'X-Api-Key': API_key_city_info})
        if response.status_code ==200:
            city_info = response.json()
            if len(city_info)>0:
                population.append(city_info[0]['population'])
                city_id.append(row["city_id"])
    
    population_df = pd.DataFrame({"population":population, "city_id": city_id})
    #population_df["city_id"] = cities_from_sql["city_id"]
    population_df["year_retrieved"] = datetime.today().strftime("%Y")
    return population_df


#To write cities information to population table
def retrieve_and_save_populations():
    population_df = create_populations_df()
    save_populations_sql(population_df)
    return "Populations appended to populations table in sql"

def save_populations_sql(population_df):
    connection_string = create_connection_string()
    population_df.to_sql('populations',
                  if_exists='append',
                  con=connection_string,
                  index=False)

retrieve_and_save_populations()

'Populations appended to populations table in sql'